In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
PATH = "https://raw.githubusercontent.com/dmitry-chvdr/aqi_prediction/master/ts_air_quality_index_2013_2020.csv"
df = pd.read_csv(PATH)

In [3]:
data_aqi_index = df[["aqi", "Date"]]
data_aqi_index = data_aqi_index.set_index("Date").sort_index()

x_train = data_aqi_index["aqi"]["2013-01-01":"2018-01-01"].values.astype("float32")
x_test = data_aqi_index["aqi"]["2018-01-02":"2020-12-31"].values.astype("float32")

In [4]:
data_aqi_index.head()

,aqi
Date,
2013-01-01,125
2013-01-02,79
2013-01-03,67
2013-01-04,62
2013-01-05,57


In [21]:
def create_dataset(dataset, features):
    x, y = [], []
    for i in range(len(dataset) - features):
        feature = dataset[i : i + features]
        target = dataset[i + 1 : i + features + 1]
        x.append(feature)
        y.append(target)
    return torch.tensor(x), torch.tensor(y)


features = 10
X_train, Y_train = create_dataset(x_train, features=features)
X_test, Y_test = create_dataset(x_test, features=features)

In [22]:
class AqiModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=10, hidden_size=100, num_layers=1, batch_first=True
        )
        self.linear = nn.Linear(100, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [36]:
model = AqiModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.MSELoss()
loader = data.DataLoader(
    data.TensorDataset(X_train, Y_train), shuffle=True, batch_size=64
)

n_epochs = 2000
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 100 != 0:
        continue

    model.eval()
    with torch.no_grad():
        y_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_pred, Y_train))
        y_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_pred, Y_test))
    print(f"Epoch: {epoch}: train RMSE: {train_rmse}, test RMSE: {test_rmse}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64, 10])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([25, 10])) that is different to the input size (torch.Size([25, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1817, 10])) that is different to the input size (torch.Size([1817, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F

Epoch: 0: train RMSE: 103.70093536376953, test RMSE: 107.17922973632812
Epoch: 100: train RMSE: 31.360572814941406, test RMSE: 34.06132888793945
Epoch: 200: train RMSE: 27.5895938873291, test RMSE: 27.999378204345703
Epoch: 300: train RMSE: 27.323637008666992, test RMSE: 27.409116744995117
Epoch: 400: train RMSE: 27.157541275024414, test RMSE: 27.070302963256836
Epoch: 500: train RMSE: 27.12288475036621, test RMSE: 27.020864486694336
Epoch: 600: train RMSE: 27.08172035217285, test RMSE: 26.953990936279297
Epoch: 700: train RMSE: 27.068099975585938, test RMSE: 26.931774139404297
Epoch: 800: train RMSE: 27.062965393066406, test RMSE: 26.969999313354492
Epoch: 900: train RMSE: 27.076845169067383, test RMSE: 26.93528175354004
Epoch: 1000: train RMSE: 27.01648712158203, test RMSE: 26.89502716064453
Epoch: 1100: train RMSE: 27.062835693359375, test RMSE: 26.932004928588867
Epoch: 1200: train RMSE: 27.006298065185547, test RMSE: 26.892562866210938
Epoch: 1300: train RMSE: 27.07609748840332, t

In [39]:
y_prediction = model(X_test).flatten().detach().numpy()

In [38]:
mean_absolute_error(x_test[:10], y_prediction[:10]), mean_squared_error(
    x_test[:10], y_prediction[:10]
)

(8.183298, 94.818535)

In [40]:
mean_absolute_error(x_test[:30], y_prediction[:30]), mean_squared_error(
    x_test[:30], y_prediction[:30]
)

(4.558486, 40.170757)

In [41]:
mean_absolute_error(x_test[:60], y_prediction[:60]), mean_squared_error(
    x_test[:60], y_prediction[:60]
)

(5.0872827, 42.98174)

LSTM с параметрами input_size=10, hidden_size=100, num_layers=1 и Adam с lr=0.001 показывает лучший результат, чем все опредыдущие модели. В дальнейшем её и будем использовать.